该代码的主要功能是从两个Excel文件中的多页数据表汇中提取学生信息，进行数据清洗与合并，并将处理后的数据保存为新的Excel文件，便于后续分析。以下是对代码的简要解释：

1. **模糊匹配函数（fuzzy_match_column）**
   用于模糊匹配字段名称，找到与目标字段最接近的列名。

2. **字段提取函数（extract_fields_from_excel）**
   从Excel文件中提取指定字段，先进行精确匹配，模糊匹配作为备选，并处理字段名称中的空格和学号字段的格式问题。返回清洗后的DataFrame。

3. **字段映射定义**
   定义了字段映射关系：

* `fields_file1`：从第一个Excel文件中提取的字段映射。
* `fields_file2`：从第二个Excel文件中提取的字段映射。

4. **数据提取与清洗**
   使用`extract_fields_from_excel`函数分别从`file1`和`file2`中提取并清洗数据，生成`df1`和`df2`。

5. **数据合并**
   使用`pd.merge`函数根据学号（`xuehao`）将两个DataFrame合并，保留在两个文件中都存在的记录。

6. **学院一致性检查**
   检查合并后的学院字段（`yuanxi_x`和`yuanxi_y`）是否一致，若不一致，输出警告并保留一个`yuanxi`列。

7. **保存合并结果**
   将合并后的数据保存为新的Excel文件。


In [ ]:
import pandas as pd
import difflib
import os

# === 路径设置 ===
base_path = r'C:\Users\31258\Desktop\新版\原数据\24级数据'
file1 = os.path.join(base_path, '【1-2】2024级新生分层数据汇总表 -全校挂牌.xls')
file2 = os.path.join(base_path, '【2-3】24-25(1)三次成绩汇总.xlsx')
output_path = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\24级学生整合成绩表.xlsx'

# === 模糊匹配函数（备选） ===
def fuzzy_match_column(keyword, columns):
    match = difflib.get_close_matches(keyword, columns, n=1, cutoff=0.3)
    return match[0] if match else None

# === 字段提取函数（优先精确匹配） ===
def extract_fields_from_excel(file_path, field_map):
    xls = pd.ExcelFile(file_path)
    all_data = pd.concat([xls.parse(sheet) for sheet in xls.sheet_names], ignore_index=True)

    all_data.columns = all_data.columns.astype(str).str.replace(r'\s+', '', regex=True)
    matched = {}
    used_columns = set()

    print(f"\n📄 正在处理文件：{file_path}")
    for zh, en in field_map.items():
        # 精确匹配
        exact_match = [col for col in all_data.columns if zh.replace(" ", "") == col]
        if exact_match:
            matched[en] = exact_match[0]
            used_columns.add(exact_match[0])
            print(f"✅ 字段精确匹配：{zh} → {exact_match[0]}")
        else:
            # 模糊匹配（备选）
            available_columns = [col for col in all_data.columns if col not in used_columns]
            fuzzy_match = fuzzy_match_column(zh, available_columns)
            if fuzzy_match:
                matched[en] = fuzzy_match
                used_columns.add(fuzzy_match)
                print(f"⚠️ 字段模糊匹配：{zh} → {fuzzy_match}")
            else:
                print(f"❌ 未找到字段: {zh}")

    df = all_data[[matched[k] for k in matched]].copy()
    df.columns = list(matched.keys())

    df = df[df['xuehao'].notna()].copy()
    df['xuehao'] = df['xuehao'].astype(str).str.strip().str.replace('.0', '', regex=False)

    return df

# === 字段映射定义 ===
fields_file1 = {
    '学号': 'xuehao',
    'xymc': 'yuanxi',
    '姓名': 'xingming',
    '高考数学规格化成绩80+(X-各类试卷均分)*折合系数': 'gaokao_math_norm',
    '入学测成绩': 'entrance_score',
    '切屏次数': 'entrance_bclass',  # ✅ 修改为实际列名
    '综合成绩(高考规格化成绩*0.7+入学测成绩*0.3)': 'total_score',
    '建议分层': 'layer_advice'
}

fields_file2 = {
    '学号': 'xuehao',
    '学院（必须填写）': 'yuanxi',
    '阶测1': 'step1',
    '阶测2': 'step2',
    '期末': 'final'
}

# === 提取数据 ===
df1 = extract_fields_from_excel(file1, fields_file1)
df2 = extract_fields_from_excel(file2, fields_file2)

# === 合并数据 ===
merged = pd.merge(df1, df2, on='xuehao', how='inner', suffixes=('_x', '_y'))

# === 保留一个 yuanxi 列，检查一致性 ===
if 'yuanxi_x' in merged.columns and 'yuanxi_y' in merged.columns:
    mismatch = (merged['yuanxi_x'] != merged['yuanxi_y']) & merged['yuanxi_x'].notna() & merged['yuanxi_y'].notna()
    if mismatch.any():
        print("\n⚠️ 学号存在学院不一致情况：")
        print(merged.loc[mismatch, ['xuehao', 'yuanxi_x', 'yuanxi_y']])
    merged['yuanxi'] = merged['yuanxi_x']
    merged.drop(['yuanxi_x', 'yuanxi_y'], axis=1, inplace=True)

# === 保存合并结果 ===
os.makedirs(os.path.dirname(output_path), exist_ok=True)
merged.to_excel(output_path, index=False)

# === 输出信息 ===
print(f"\n✅ 合并成功，最终记录数：{len(merged)} 条")
print(f"📄 合并结果保存至：{output_path}")



📄 正在处理文件：C:\Users\31258\Desktop\新版\原数据\24级数据\【1-2】2024级新生分层数据汇总表 -全校挂牌.xls
✅ 字段精确匹配：学号 → 学号
✅ 字段精确匹配：xymc → xymc
✅ 字段精确匹配：姓名 → 姓名
✅ 字段精确匹配：高考数学规格化成绩80+(X-各类试卷均分)*折合系数 → 高考数学规格化成绩80+(X-各类试卷均分)*折合系数
✅ 字段精确匹配：入学测成绩 → 入学测成绩
✅ 字段精确匹配：切屏次数 → 切屏次数
✅ 字段精确匹配：综合成绩(高考规格化成绩*0.7+入学测成绩*0.3) → 综合成绩(高考规格化成绩*0.7+入学测成绩*0.3)
⚠️ 字段模糊匹配：建议分层 → 分层

📄 正在处理文件：C:\Users\31258\Desktop\新版\原数据\24级数据\【2-3】24-25(1)三次成绩汇总.xlsx
✅ 字段精确匹配：学号 → 学号
✅ 字段精确匹配：学院（必须填写） → 学院（必须填写）
⚠️ 字段模糊匹配：阶测1 → 阶测1成绩
⚠️ 字段模糊匹配：阶测2 → 阶测2成绩
⚠️ 字段模糊匹配：期末 → 期末成绩

⚠️ 学号存在学院不一致情况：
            xuehao yuanxi_x    yuanxi_y
890   202483270165     长望学院   电子与信息工程学院
891   202483270165     长望学院   电子与信息工程学院
930   202483270070     长望学院   电子与信息工程学院
931   202483270070     长望学院   电子与信息工程学院
1216  202413930097     长望学院       自动化学院
...            ...      ...         ...
6174  202483270143   管理工程学院  电子与信息工程 学院
6175  202483270143   管理工程学院  电子与信息工程 学院
6252  202483390047  化学与材料学院      管理工程学院
6269  202483270519      商学院   电子与信息工程学院
6274  202413930089      商学院      

该代码的核心功能是从原始数据中清洗出有效的学生成绩信息，删除不符合要求的记录，并将清洗后的数据保存为新文件。以下是对核心逻辑的说明：

1. **加载数据**
   通过`pd.read_excel()`加载原始的Excel数据，获取学生成绩的完整记录，并统计原始数据总条数。

2. **确保成绩字段为数值型**
   对`step1`、`step2`、`final`等成绩字段进行转换，确保它们为数值型数据。对于非数字或无法转换为数字的值，使用`NaN`填充。

3. **删除成绩为0、空值或非数字的记录**
   对每个成绩字段（如`gaokao_math_norm`、`entrance_score`等），删除其中成绩为0、空值或非数字的记录。删除时打印被删除记录的数量及前3条示例，确保用户了解数据处理情况。

4. **删除切屏次数 ≥ 6 的记录**
   将`entrance_bclass`字段转换为数值型，并删除切屏次数（`entrance_bclass`）大于等于6的记录。也打印出被删除的记录数量及示例，便于确认。

5. **删除分层结果为空或空字符串的记录**
   删除分层结果（`layer_advice`）为空或空字符串的记录。确保分层数据的完整性，以便后续分析。

6. **结果统计与保存**
   通过`df.reset_index()`重新索引，并计算清洗后的数据总条数。最终将清洗后的数据保存为新的Excel文件，输出处理信息，包括清洗后的记录数和删除的记录数。

总结：
此代码通过清洗操作，去除了无效的成绩记录、切屏次数过多的记录以及分层结果为空的记录，保证了数据的质量，最终生成了一个清洗后的学生成绩数据集。

In [13]:
import pandas as pd

# === 文件路径设置 ===
file_path = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\24级学生整合成绩表.xlsx'
output_path = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\24级学生整合成绩表_清洗后.xlsx'

# === 加载原始数据 ===
df = pd.read_excel(file_path)
original_count = len(df)

print(f"📥 原始数据总记录数：{original_count} 条")

# === 成绩字段定义 ===
score_cols = ['gaokao_math_norm', 'entrance_score', 'total_score', 'step1', 'step2', 'final']

# === 确保 step1、step2、final 为数值型（非数字转 NaN）===
for col in ['step1', 'step2', 'final']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# === 步骤 1：删除成绩字段为 0、空值或非数字 的行 ===
for col in score_cols:
    before = len(df)
    condition = df[col].isna() | (df[col] == 0)
    removed = df[condition]
    df = df[~condition]
    print(f"🔍 删除【{col}】为 0 或空值的记录：{len(removed)} 条")
    if not removed.empty:
        print(removed[['xuehao', col]].head(3))  # 打印前3条示例

# === 步骤 2：删除切屏次数 ≥ 6 的行 ===
df['entrance_bclass'] = pd.to_numeric(df['entrance_bclass'], errors='coerce')
condition = df['entrance_bclass'] >= 6
removed = df[condition]
df = df[~condition]
print(f"🔍 删除切屏次数 ≥ 6 的记录：{len(removed)} 条")
if not removed.empty:
    print(removed[['xuehao', 'entrance_bclass']].head(3))

# === 步骤 3：删除分层结果 layer_advice 为空 或 空字符串 的行 ===
condition = df['layer_advice'].isna() | (df['layer_advice'].astype(str).str.strip() == '')
removed = df[condition]
df = df[~condition]
print(f"🔍 删除分层结果为空的记录：{len(removed)} 条")
if not removed.empty:
    print(removed[['xuehao', 'layer_advice']].head(3))

# === 结果统计 ===
df.reset_index(drop=True, inplace=True)
final_count = len(df)
print(f"\n✅ 清洗完成！剩余记录数：{final_count} 条（共删除 {original_count - final_count} 条）")

# === 保存清洗结果 ===
df.to_excel(output_path, index=False)
print(f"📄 清洗后的文件已保存至：{output_path}")


📥 原始数据总记录数：6276 条
🔍 删除【gaokao_math_norm】为 0 或空值的记录：62 条
           xuehao  gaokao_math_norm
846  202483300459               0.0
847  202483300291               0.0
848  202483300268               0.0
🔍 删除【entrance_score】为 0 或空值的记录：237 条
           xuehao  entrance_score
750  202483300096             0.0
800  202483300293             0.0
803  202483300236             0.0
🔍 删除【total_score】为 0 或空值的记录：0 条
🔍 删除【step1】为 0 或空值的记录：13 条
            xuehao  step1
202   202483300689    NaN
1092  202483450105    NaN
1291  202483450060    NaN
🔍 删除【step2】为 0 或空值的记录：24 条
            xuehao  step2
214   202483300231    NaN
376   202483300306    NaN
1004  202483230036    NaN
🔍 删除【final】为 0 或空值的记录：9 条
            xuehao  final
98    202483300653    NaN
1633  202483230032    NaN
1910  202483290035    NaN
🔍 删除切屏次数 ≥ 6 的记录：315 条
           xuehao  entrance_bclass
377  202483300798               13
378  202483300433                6
379  202483300479               13
🔍 删除分层结果为空的记录：2036 条
           xuehao l

根据合并、清洗好的数据，进行随机抽样调查。将'自动化学院', '龙山书院', '电子与信息工程学院'三个学院中全部的学生数据分别拆分到三个Excel表文件中，用于后续的分析。

In [14]:
import pandas as pd
import os

# ✅ 已清洗后的 Excel 文件路径（根据实际路径修改）
cleaned_file_path = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\24级学生整合成绩表_清洗后.xlsx'

# ✅ 加载数据
df_cleaned = pd.read_excel(cleaned_file_path)

# ✅ 要提取的目标院系列表
target_departments = ['自动化学院', '龙山书院', '电子与信息工程学院']

# ✅ 院系列字段名（如果你的列名不是 'yuanxi' 请替换）
department_col = 'yuanxi'

# ✅ 保存目录（与源文件同目录）
output_dir = os.path.join(os.path.dirname(cleaned_file_path), '按院系拆分')
os.makedirs(output_dir, exist_ok=True)

# ✅ 遍历每个院系并保存
for dept in target_departments:
    dept_df = df_cleaned[df_cleaned[department_col] == dept]

    if not dept_df.empty:
        save_path = os.path.join(output_dir, f'{dept}_成绩表.xlsx')
        dept_df.to_excel(save_path, index=False)
        print(f'✅ 已保存：{dept}（{len(dept_df)} 条记录） → {save_path}')
    else:
        print(f'⚠️ 未找到数据：{dept}')


✅ 已保存：自动化学院（612 条记录） → C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\按院系拆分\自动化学院_成绩表.xlsx
✅ 已保存：龙山书院（792 条记录） → C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\按院系拆分\龙山书院_成绩表.xlsx
✅ 已保存：电子与信息工程学院（726 条记录） → C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\按院系拆分\电子与信息工程学院_成绩表.xlsx


# 代码功能总结

这段代码主要实现了对抽取出来的三个院系的学生成绩数据进行相关性分析，并生成相关性分析报告和可视化图表。具体过程如下：

1. **数据加载与院系分表读取**  
   代码首先从指定路径读取清洗后的学生成绩数据，并自动查找每个院系的成绩表。对于每个院系的数据，都会单独进行分析。

2. **相关性分析**  
   代码定义了多个字段对，进行皮尔逊相关性分析，计算各个字段之间的相关系数和p值。分析的字段包括高考数学规范化成绩、入学测成绩、阶段性成绩（1和2）和期末成绩。

3. **结果记录与输出**  
   对每个院系，代码会计算相关系数和p值，并根据p值判断相关性是否显著。如果样本数据不足，标记为“样本过少”。分析结果会被存储到一个Excel报告中，并同时为每个相关性字段对生成对应的散点图。

4. **图表保存**  
   相关性分析结果通过`seaborn`绘制散点图和回归线，并将图表保存在指定的文件夹中，确保每个图表的文件名合法。

最终，代码完成后会输出分析完成的提示，报告和图表被保存在指定的输出目录，方便用户查看和分析。

In [15]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr
import matplotlib

# === 设置 matplotlib 中文支持 ===
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False

# === 路径设置 ===
base_path = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级'
input_path = os.path.join(base_path, '24级学生整合成绩表_清洗后.xlsx')

# 院系分表路径（自动查找）
dept_dir = os.path.join(base_path, '按院系拆分')

# 输出结果目录
output_dir = r'C:\Users\31258\Desktop\新版\分析\24级\相关性'
os.makedirs(output_dir, exist_ok=True)

# === 读取总数据 ===
df_all = pd.read_excel(input_path)

# === 院系字段名 ===
department_col = 'yuanxi'

# === 自动化读取部门文件 ===
target_departments = ['自动化学院', '龙山书院', '电子与信息工程学院']
departments = {'全体': df_all}

for dept in target_departments:
    dept_file = os.path.join(dept_dir, f'{dept}_成绩表.xlsx')
    if os.path.exists(dept_file):
        departments[dept] = pd.read_excel(dept_file)
    else:
        print(f'⚠️ 未找到文件: {dept_file}')

# === 要分析的字段对 ===
correlation_pairs = [
    ('gaokao_math_norm', 'entrance_score'),
    ('step1', 'step2'),
    ('step1', 'final'),
    ('step2', 'final')
]

# === 存储所有分析结果 ===
results_all = []

# === 分析函数 ===
def analyze_and_plot(df, dept_name):
    for x_col, y_col in correlation_pairs:
        data = df[[x_col, y_col]].dropna()
        if len(data) < 3:
            results_all.append({
                '院系': dept_name,
                '变量1': x_col,
                '变量2': y_col,
                '样本数': len(data),
                '相关系数 r': None,
                'p 值': None,
                '备注': '样本过少'
            })
            continue

        r, p = pearsonr(data[x_col], data[y_col])
        results_all.append({
            '院系': dept_name,
            '变量1': x_col,
            '变量2': y_col,
            '样本数': len(data),
            '相关系数 r': round(r, 4),
            'p 值': format(p, '.4g'),
            '备注': '显著' if p < 0.05 else '不显著'
        })

        # 绘图
        plt.figure(figsize=(6, 4))
        sns.regplot(x=x_col, y=y_col, data=data, line_kws={"color": "red"})
        plt.title(f'{dept_name}: {x_col} vs {y_col}\nr = {r:.4f}, p = {p:.4g}')
        plt.xlabel(x_col)
        plt.ylabel(y_col)
        plt.tight_layout()

        # 合法文件名
        safe_name = f"{dept_name}_{x_col}_vs_{y_col}".replace(":", "").replace("/", "_")
        plt.savefig(os.path.join(output_dir, f"{safe_name}.png"))
        plt.close()

# === 执行分析 ===
for dept_name, df in departments.items():
    analyze_and_plot(df, dept_name)

# === 保存分析结果 ===
results_df = pd.DataFrame(results_all)
excel_output = os.path.join(output_dir, '相关性分析报告.xlsx')
results_df.to_excel(excel_output, index=False)

# === 完成提示 ===
print("✅ 分析完成")
print("📄 报告已保存至：", excel_output)
print("🖼️ 图表保存目录：", output_dir)


✅ 分析完成
📄 报告已保存至： C:\Users\31258\Desktop\新版\分析\24级\相关性\相关性分析报告.xlsx
🖼️ 图表保存目录： C:\Users\31258\Desktop\新版\分析\24级\相关性


这段代码的主要作用是针对不同院系的学生成绩数据，进行分层均值检验（t检验），分析不同分层（A层与B层）之间在各个成绩字段（如step1, step2, final）上的差异，且根据检验结果生成柱状图和箱线图，最终保存分析结果和图表。（经分析后这个结果有问题，问题来源于数据清洗的规则）

In [16]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import matplotlib

# ✅ 设置中文字体支持
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False

# === 分析字段 ===
test_columns = ['step1', 'step2', 'final']
layer_col = 'layer_advice'

# === 数据路径（清洗后的24级） ===
data_path = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\24级学生整合成绩表_清洗后.xlsx'
df_all = pd.read_excel(data_path)

# === 院系字段 ===
department_col = 'yuanxi'
departments = {
    '全体': df_all,
    '自动化学院': df_all[df_all[department_col] == '自动化学院'],
    '龙山书院': df_all[df_all[department_col] == '龙山书院'],
    '电子与信息工程学院': df_all[df_all[department_col] == '电子与信息工程学院']
}

# === 输出路径 ===
base_output_dir = r'C:\Users\31258\Desktop\新版\分析\24级\分层均值检验'
os.makedirs(base_output_dir, exist_ok=True)

# === 存放 t 检验结果 ===
t_results = []

# === 分析函数 ===
def test_layer_and_plot(df, dept_name):
    dept_dir = os.path.join(base_output_dir, dept_name)
    os.makedirs(dept_dir, exist_ok=True)

    for col in test_columns:
        sub = df[[col, layer_col]].dropna()

        if sub[layer_col].nunique() < 2:
            t_results.append({
                '院系': dept_name,
                '变量': col,
                'A层均值': None,
                'B层均值': None,
                'p值': None,
                '结论': '无A/B层或数据不足'
            })
            continue

        A = sub[sub[layer_col] == 'A'][col]
        B = sub[sub[layer_col] == 'B'][col]

        if len(A) < 2 or len(B) < 2:
            t_results.append({
                '院系': dept_name,
                '变量': col,
                'A层均值': A.mean() if len(A) > 0 else None,
                'B层均值': B.mean() if len(B) > 0 else None,
                'p值': None,
                '结论': '样本数不足'
            })
            continue

        # ✅ t检验：A > B 是否显著
        t_stat, p_val = ttest_ind(A, B, equal_var=False, alternative='greater')
        t_results.append({
            '院系': dept_name,
            '变量': col,
            'A层均值': round(A.mean(), 2),
            'B层均值': round(B.mean(), 2),
            'p值': format(p_val, '.4g'),
            '结论': 'A显著高于B' if p_val < 0.05 else '差异不显著'
        })

        # ✅ 柱状图
        plt.figure(figsize=(5, 4))
        sns.barplot(data=sub, x=layer_col, y=col, hue=layer_col, errorbar='sd', palette='pastel', legend=False)
        plt.title(f'{dept_name}：{col} 平均成绩对比（A vs B）')
        plt.xlabel("分层")
        plt.ylabel(col)
        plt.tight_layout()
        plt.savefig(os.path.join(dept_dir, f'{col}_bar.png'))
        plt.close()

        # ✅ 箱线图
        plt.figure(figsize=(5, 4))
        sns.boxplot(data=sub, x=layer_col, y=col, hue=layer_col, palette='Set2', legend=False)
        plt.title(f'{dept_name}：{col} 成绩分布（箱线图）')
        plt.xlabel("分层")
        plt.ylabel(col)
        plt.tight_layout()
        plt.savefig(os.path.join(dept_dir, f'{col}_box.png'))
        plt.close()

# === 执行所有院系分析 ===
for dept, df in departments.items():
    test_layer_and_plot(df, dept)

# === 保存最终检验结果表 ===
output_file = os.path.join(base_output_dir, '分层均值检验结果.xlsx')
pd.DataFrame(t_results).to_excel(output_file, index=False)

# === 提示信息 ===
print("✅ 所有院系均值检验与图表保存完成")
print("📊 检验表格：", output_file)
print("🖼️ 图表文件夹：", base_output_dir)


✅ 所有院系均值检验与图表保存完成
📊 检验表格： C:\Users\31258\Desktop\新版\分析\24级\分层均值检验\分层均值检验结果.xlsx
🖼️ 图表文件夹： C:\Users\31258\Desktop\新版\分析\24级\分层均值检验


# 数据清洗规则（新）

以下是根据上一个分层均值检验中问题进行分析，重新构建的数据清洗过程的规则：

1. **字段选择**  
   只保留必要的字段：`xuehao`, `yuanxi`, `step1`, `step2`, `final`, `layer_advice`，其余不相关的字段被剔除，减少数据量，确保只处理所需信息。

2. **数值转换**  
   将成绩字段（`step1`, `step2`, `final`）转换为数值型。如果这些字段中的值无法转换为数值（例如包含文本或空值），则会被替换为 `NaN`。这保证了数据的统一性，排除无效字段。

3. **异常成绩过滤**  
   删除成绩小于 0 或大于 100 的记录。因为学生成绩通常应在此范围内，任何超出此范围的值被视为异常数据，因此会被移除，确保数据合理性。

4. **删除空的分层结果**  
   删除 `layer_advice`（分层结果）为空或仅包含空格的记录。通过 `notna()` 检查非 `NaN` 值，并通过 `str.strip()` 去除字符串的空格，确保仅保留有效的分层信息。

5. **索引重置**  
   在删除无效记录后，重置数据的索引。使用 `reset_index(drop=True, inplace=True)` 重新编号，使数据的索引从 0 开始，并丢弃旧的索引。

6. **保存清洗后的数据**  
   将清洗后的数据保存到新的Excel文件中，便于后续处理和分析，并输出清洗后的记录数和保存路径。

这些规则帮助确保数据质量、提高数据一致性，为后续的数据分析做好准备。

In [5]:
import pandas as pd
import os

# === 文件路径设置 ===
base_dir = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级'
input_file = os.path.join(base_dir, '24级学生整合成绩表.xlsx')
output_file = os.path.join(base_dir, '24级学生整合成绩_分层检验用数据.xlsx')

# === 加载原始数据 ===
df = pd.read_excel(input_file)
print(f"📥 读取成功，共 {len(df)} 条记录")

# === 提取目标字段 ===
columns_needed = ['xuehao', 'yuanxi', 'step1', 'step2', 'final', 'layer_advice']
df_selected = df[columns_needed].copy()

# === 转换成绩字段为数值型（处理非数字）===
for col in ['step1', 'step2', 'final']:
    df_selected[col] = pd.to_numeric(df_selected[col], errors='coerce')

# === 清洗异常成绩（小于 0 或 大于 100）===
for col in ['step1', 'step2', 'final']:
    df_selected = df_selected[(df_selected[col] >= 0) & (df_selected[col] <= 100)]

# === 删除分层结果为空的行 ===
df_selected = df_selected[
    df_selected['layer_advice'].notna() &
    (df_selected['layer_advice'].astype(str).str.strip() != '')
]

# === 重置索引 ===
df_selected.reset_index(drop=True, inplace=True)

# === 保存清洗后的数据 ===
df_selected.to_excel(output_file, index=False)
print(f"✅ 清洗完成，共保留 {len(df_selected)} 条记录")
print(f"📄 文件已保存至：{output_file}")


📥 读取成功，共 6276 条记录
✅ 清洗完成，共保留 3965 条记录
📄 文件已保存至：C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\24级学生整合成绩_分层检验用数据.xlsx


随机抽样，抽取三个学院（'自动化学院', '龙山书院', '计算机学院'）的数据进行分层均值检验，验证分班的科学性。

In [8]:
import pandas as pd
import os

# ✅ 已清洗后的 Excel 文件路径（根据实际路径修改）
cleaned_file_path = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\24级学生整合成绩_分层检验用数据.xlsx'

# ✅ 加载数据
df_cleaned = pd.read_excel(cleaned_file_path)

# ✅ 要提取的目标院系列表
target_departments = ['自动化学院', '龙山书院', '计算机学院']

# ✅ 院系列字段名（如果你的列名不是 'yuanxi' 请替换）
department_col = 'yuanxi'

# ✅ 保存目录（与源文件同目录）
output_dir = os.path.join(os.path.dirname(cleaned_file_path), '按院系拆分')
os.makedirs(output_dir, exist_ok=True)

# ✅ 遍历每个院系并保存
for dept in target_departments:
    dept_df = df_cleaned[df_cleaned[department_col] == dept]

    if not dept_df.empty:
        save_path = os.path.join(output_dir, f'{dept}_成绩表.xlsx')
        dept_df.to_excel(save_path, index=False)
        print(f'✅ 已保存：{dept}（{len(dept_df)} 条记录） → {save_path}')
    else:
        print(f'⚠️ 未找到数据：{dept}')


✅ 已保存：自动化学院（651 条记录） → C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\按院系拆分\自动化学院_成绩表.xlsx
✅ 已保存：龙山书院（869 条记录） → C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\按院系拆分\龙山书院_成绩表.xlsx
✅ 已保存：计算机学院（514 条记录） → C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\按院系拆分\计算机学院_成绩表.xlsx


# 代码功能总结

该代码的主要功能是对全校学生以及抽取的三个院系的学生成绩数据进行分层均值检验（t检验），并生成相关的可视化图表。具体步骤如下：

1. **数据加载与准备：**
   - 读取了清洗后的学生成绩数据文件 `24级学生整合成绩_分层检验用数据.xlsx`。
   - 提取了必要的字段，包括成绩字段 (`step1`, `step2`, `final`) 和分层字段 (`layer_advice`)。
   - 对各院系的数据进行了分组，包括“全体”、“自动化学院”、“龙山书院”和“计算机学院”。

2. **分层均值检验：**
   - 对每个院系的数据，针对每个成绩字段（`step1`, `step2`, `final`），进行分层（A层与B层）的 t 检验。
   - 使用 `ttest_ind` 进行独立样本 t 检验，检验A层与B层的成绩是否存在显著差异。
   - 如果某一层的数据不足（如样本数小于2），则跳过该分析，并记录“样本数不足”。
   - 对每个字段的检验结果（包括A层均值、B层均值、p值和结论）进行了存储。

3. **数据可视化：**
   - 对每个院系，生成了柱状图和箱线图，展示A层与B层在各个成绩字段上的均值对比及成绩分布情况。
   - 使用 `seaborn` 绘制了具有回归线的柱状图和箱线图，图表文件保存到相应院系的文件夹中。

4. **输出结果：**
   - 将所有院系的 t 检验结果保存到 `分层均值检验结果.xlsx` 文件中，并将图表保存到指定的文件夹中。
   - 输出了完成提示，显示检验结果文件和图表文件夹的路径。

### 代码流程：
- **数据加载：** 读取清洗后的学生成绩数据。
- **分层均值检验：** 针对不同院系和成绩字段，进行 t 检验并生成结果。
- **图表生成：** 为每个成绩字段生成柱状图和箱线图。
- **结果保存：** 将检验结果保存为 Excel 文件，图表保存为图片。

### 主要技术点：
- 使用 `ttest_ind` 进行 t 检验。
- 使用 `seaborn` 和 `matplotlib` 绘制柱状图和箱线图。
- 使用 `pandas` 进行数据处理、清洗和保存。

In [10]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
import matplotlib

# ✅ 设置中文字体支持
matplotlib.rcParams['font.sans-serif'] = ['SimHei']
matplotlib.rcParams['axes.unicode_minus'] = False

# === 分析字段 ===
test_columns = ['step1', 'step2', 'final']
layer_col = 'layer_advice'

# === 数据路径（清洗后的24级） ===
data_path = r'C:\Users\31258\Desktop\新版\数据整合以及清洗\24级\24级学生整合成绩_分层检验用数据.xlsx'
df_all = pd.read_excel(data_path)

# === 院系字段 ===
department_col = 'yuanxi'
departments = {
    '全体': df_all,
    '自动化学院': df_all[df_all[department_col] == '自动化学院'],
    '龙山书院': df_all[df_all[department_col] == '龙山书院'],
    '计算机学院': df_all[df_all[department_col] == '计算机学院']
}

# === 输出路径 ===
base_output_dir = r'C:\Users\31258\Desktop\新版\分析\24级\分层均值检验'
os.makedirs(base_output_dir, exist_ok=True)

# === 存放 t 检验结果 ===
t_results = []

# === 分析函数 ===
def test_layer_and_plot(df, dept_name):
    dept_dir = os.path.join(base_output_dir, dept_name)
    os.makedirs(dept_dir, exist_ok=True)

    for col in test_columns:
        sub = df[[col, layer_col]].dropna()

        if sub[layer_col].nunique() < 2:
            t_results.append({
                '院系': dept_name,
                '变量': col,
                'A层均值': None,
                'B层均值': None,
                'p值': None,
                '结论': '无A/B层或数据不足'
            })
            continue

        A = sub[sub[layer_col] == 'A'][col]
        B = sub[sub[layer_col] == 'B'][col]

        if len(A) < 2 or len(B) < 2:
            t_results.append({
                '院系': dept_name,
                '变量': col,
                'A层均值': A.mean() if len(A) > 0 else None,
                'B层均值': B.mean() if len(B) > 0 else None,
                'p值': None,
                '结论': '样本数不足'
            })
            continue

        # ✅ t检验：A > B 是否显著
        t_stat, p_val = ttest_ind(A, B, equal_var=False, alternative='greater')
        t_results.append({
            '院系': dept_name,
            '变量': col,
            'A层均值': round(A.mean(), 2),
            'B层均值': round(B.mean(), 2),
            'p值': format(p_val, '.4g'),
            '结论': 'A显著高于B' if p_val < 0.05 else '差异不显著'
        })

        # ✅ 柱状图
        plt.figure(figsize=(5, 4))
        sns.barplot(data=sub, x=layer_col, y=col, hue=layer_col, errorbar='sd', palette='pastel', legend=False)
        plt.title(f'{dept_name}：{col} 平均成绩对比（A vs B）')
        plt.xlabel("分层")
        plt.ylabel(col)
        plt.tight_layout()
        plt.savefig(os.path.join(dept_dir, f'{col}_bar.png'))
        plt.close()

        # ✅ 箱线图
        plt.figure(figsize=(5, 4))
        sns.boxplot(data=sub, x=layer_col, y=col, hue=layer_col, palette='Set2', legend=False)
        plt.title(f'{dept_name}：{col} 成绩分布（箱线图）')
        plt.xlabel("分层")
        plt.ylabel(col)
        plt.tight_layout()
        plt.savefig(os.path.join(dept_dir, f'{col}_box.png'))
        plt.close()

# === 执行所有院系分析 ===
for dept, df in departments.items():
    test_layer_and_plot(df, dept)

# === 保存最终检验结果表 ===
output_file = os.path.join(base_output_dir, '分层均值检验结果.xlsx')
pd.DataFrame(t_results).to_excel(output_file, index=False)

# === 提示信息 ===
print("✅ 所有院系均值检验与图表保存完成")
print("📊 检验表格：", output_file)
print("🖼️ 图表文件夹：", base_output_dir)


✅ 所有院系均值检验与图表保存完成
📊 检验表格： C:\Users\31258\Desktop\新版\分析\24级\分层均值检验\分层均值检验结果.xlsx
🖼️ 图表文件夹： C:\Users\31258\Desktop\新版\分析\24级\分层均值检验
